### Do the imports
- If I am doing any processing of files in a folder I like to import os and glob
- import cellpose models for training of inferance
- import something to read your images 
- import libraries to save your results


### Install pytorch if it isn't installed yeet

Use this website to configure the right installation packages:
[Pytorch](https://pytorch.org/get-started/locally/)

then use `pip install <list of packages>` to install it

For my macbook, it is

```pip3 install torch torchvision torchaudio```



In [72]:
import os
import glob
import sys

import tifffile # read an write tiff files
from cellpose import models
import napari
import torch
import nd2
import numpy as np
from skimage.filters import gaussian

from skimage.transform import rescale
import skimage as ski

import utils

Create the napari viewer to visualize images and mask

In [2]:
viewer = napari.Viewer()

Find the image files to work with. Different path for Window, Mac, and Linux

In [3]:
if sys.platform == 'darwin': 
    imagedir = "/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data"
elif sys.plaform == 'win32':
    imagedir = os.path.join(['S:', 'micro',
                             'internal', 'cjw',
                             'PythonImageProcessing',
                             'cellpose',
                             'Data'])
else:
   imagedir = "/n/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data"
   
imagefiles = sorted(glob.glob(f"{imagedir}/*.tif"))
imagefiles

['/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/01-yeast_many.tif',
 '/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/02-a01_42.tif',
 '/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/03-a01_43.tif',
 '/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/04-nuclei_fish.tif',
 '/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/05-yeast_01.tif',
 '/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/06-yeast_02.tif',
 '/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/C-hela-cells.tif',
 '/Volumes/core/micro/internal/cjw/PythonImageProcessing/cellpose/Data/fluo_00.tif']

In [53]:
cells = tifffile.imread(imagefiles[-1])
cells.shape

(3, 1030, 1300)

In [58]:
lysosomes = cells[0]
mitocondria = cells[1]
nuclei = cells[2]

Add the image to the napari viewer

In [59]:
viewer.layers.clear()
viewer.add_image(lysosomes, colormap='magenta', blending='additive')
viewer.add_image(mitocondria, colormap='green', blending='additive')
viewer.add_image(nuclei, colormap='bop blue', blending='additive')



<Image layer 'nuclei' at 0x2f3fe03a0>

### Setup the cellpose model

#### GPU setting
- on windows or linux with an NVIDIA GPU, set `gpu=True``
- on M1 or M2 mac, set `device=torch.device('mps')`
- on old Mac or Windows without GPU, set `gpu=False` -- this will be slower

#### Model
- cellpose has a number of pretrained models to use. Start with `cyto` or `cyto2` (even it is nuclei)
- info about pretrained cellpose [models](https://cellpose.readthedocs.io/en/latest/models.html)

In [85]:
d = torch.device('mps')

# change gpu=True is on windows, and get rid of device
# model = models.Cellpose(gpu=True, model_type='cyto2')
model = models.Cellpose(gpu=False, device=d, model_type='nuclei')
cyto = models.Cellpose(gpu=False, device=d, model_type='cyto2')

### `model.eval`
Parameters
----------

The basic options:
- x : the image, can be a 2d numpy array, a list of numpy arrays,
or a 3d numpy array
- diameter : The approximate size of the object you are trying to segment
- channels :
    - [0, 0] for a grayscale image
    - [] 

Returns
-------
- masks : An array or list of arrays with segmenation labels/masks
- flows : A list of numpy arrays with fow

In [74]:
bs_nuc = ski.morphology.white_tophat(nuclei, ski.morphology.disk(25))

In [75]:
viewer.add_image(bs_nuc)

<Image layer 'bs_nuc' at 0x32663ba00>

In [83]:

masks, flows, styles, diams = model.eval(nuclei, channels=[0, 0],   #[0, 3] might work best
                                         diameter=50)

In [104]:
mitonuc = np.stack([mitocondria, bs_nuc])
mito_masks, flows, styles, diams = cyto.eval(mitonuc, channels=[1, 2],   #[0, 3] might work best
                                         diameter=75)

In [109]:
lysonuc = np.stack([lysosomes, bs_nuc])
lyto_masks, flows, styles, diams = cyto.eval(lysonuc, channels=[1, 2],   #[0, 3] might work best
                                         diameter=75)

Show the results from cellpose a layers in napari

In [110]:
#viewer.layers.clear()
clear = True
if clear:
    rlist =list()
    for idx, y in enumerate(viewer.layers):
        if 'mask' in y.name:
            rlist.append(y)
    for r in rlist:
        viewer.layers.remove(r)
viewer.add_labels(masks, name='nuclei_mask')
viewer.add_labels(mito_masks)
viewer.add_labels(lyto_masks)


<Labels layer 'lyto_masks' at 0x39e015330>

In [98]:
for y in viewer.layers:
        print(y.name)

lysosomes
mitocondria
nuclei
Image
bs_nuc
nuclei_mask
nuclei_mask [1]
mito_masks


In [41]:
g_mito = gaussian(mitocondria, 3, preserve_range=True)
g_lyso = gaussian(lysosomes, 3, preserve_range=True)

In [50]:
lysonuc = np.stack([g_lyso, nuclei])
lyso_mask, flows, styles, diams = model.eval(g_lyso, channels=[1, 2],   #[0, 3] might work best
                                         diameter=200)

mitonuc = np.stack([g_mito, nuclei])
mito_mask, flows, styles, diams = model.eval(g_mito, channels=[1, 2],   #[0, 3] might work best
                                         diameter=200,
                                         )

In [51]:
clear = False
if clear:
    for y in viewer.layers:
        if 'mask' in y.name:
            viewer.layers.remove(y)

viewer.add_labels(lyso_mask)
viewer.add_labels(mito_mask)

<Labels layer 'mito_mask [1]' at 0x2f374d4b0>

In [37]:
viewer.add_image(flows[0], visible=False, name="xyflow")
viewer.add_image(flows[1][0], visible=False, name="xflow")
viewer.add_image(flows[2], visible=False, name="probability")

<Image layer 'probability' at 0x30e8bfca0>

See what happens when the diameter is changed. What diameter is the best?

In [52]:
viewer.add_image(g_lyso, colormap='magenta')
viewer.add_image(g_mito, colormap='green')

<Image layer 'g_mito' at 0x2f5019360>